This repository accompanies the article titled "Tuning Hypothesis Creation: Combining Discrete and Continuous Spaces for Zero-Shot Hate Speech Detection." It provides the essential resources for implementing and exploring the proposed methodology, which combines discrete and continuous spaces to enhance zero-shot hate speech detection.

# Pretrained Weights for Virtual Tokens:

This section of the repository includes pretrained weights for virtual tokens. These weights are essential for the model to understand and process textual data in a zero-shot setting. They have been fine-tuned to excel in hate speech detection tasks.

# Jupyter Notebook:

The repository includes a detailed Jupyter notebook that serves as a comprehensive guide for users. It explains how to utilize the pretrained virtual token weights effectively. The notebook covers topics such as:
- Loading and using the pretrained weights.
- Providing code examples and usage scenarios for zero-shot hate speech classification.

# Imports

In [72]:
import torch as t
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd 

# Load Pretrained Roberta

In [34]:
m_name = 'roberta-large-mnli'

In [36]:
model = AutoModelForSequenceClassification.from_pretrained(m_name, num_labels=3).cuda()
tokenizer = AutoTokenizer.from_pretrained(m_name)

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

C:\Users\Lorenzo\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lorenzo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassif

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Create hypothesis vectors

In [65]:
# Pretrained hypothesis
full_optimized = t.load("VIRTUAL_TOKENS.pt").detach()

In [95]:
hateful_or_effensive = tokenizer.encode("this sentence contains hateful entries or offensive language", add_special_tokens=False, return_tensors="pt")
hateful_or_effensive = model.cuda().base_model.embeddings.word_embeddings(hateful_or_effensive.cuda()).reshape([-1, 1024])

# Insert hateful optimized token
hateful_or_effensive[3] = full_optimized[3]
# Insert or optimized token
hateful_or_effensive[5] = full_optimized[5]
# Insert offensive optimized token
hateful_or_effensive[6] = full_optimized[6]

In [86]:
# Standard hypothesis
pure_hypothesis1 = tokenizer.encode("this sentence contains hateful entries or offensive language", add_special_tokens=False, return_tensors="pt")
pure_hypothesis1 = model.cuda().base_model.embeddings.word_embeddings(pure_hypothesis1.cuda()).reshape([-1, 1024])

pure_hypothesis2 = tokenizer.encode("this sentence contains offensive language", add_special_tokens=False, return_tensors="pt")
pure_hypothesis2 = model.cuda().base_model.embeddings.word_embeddings(pure_hypothesis2.cuda()).reshape([-1,1024])

# Custom create entailment calcualtion

In [87]:
sep_token_emb = model.cuda().base_model.embeddings.word_embeddings(t.tensor(tokenizer.sep_token_id).int().cuda())

def comput_entailment(test,v_tokens_emb):
    
    input = t.tensor(tokenizer(test)["input_ids"]).cuda()

    add_virtual_tokens = t.cat([
            sep_token_emb.reshape(1,-1).cuda(),
            v_tokens_emb,
            sep_token_emb.reshape(1,-1).cuda()
        ],dim=0)

    words_emb = model.base_model.embeddings.word_embeddings(input)

    model_input_emb = t.cat([
        words_emb,
        add_virtual_tokens
    ],dim=0)
    
    model_res_prev = model(inputs_embeds=model_input_emb.reshape(1,-1,1024)).logits.detach().cpu()
    return t.nn.functional.softmax(model_res_prev, dim=1).reshape(-1)

# Test

In [109]:
save_res = []

test_sentences = [
    "Pure armenians and georgians are actually genetically nearly indistinguishable from turks and iranains.",
    "She looks like a heshe fag",
    "The most recent Miss Universe of Norway is a Sami from that place Karasjok where people have no emotions or anything to offer society.",
    "Decent people are allowed here, so queers keep out"
                ]

for test_sentence in test_sentences:

    for hypo in zip([full_optimized, 
                     hateful_or_effensive,
                     pure_hypothesis1, 
                     pure_hypothesis2],
                    ["full_optimized",
                     "this sentence contains [Tuned HATEFUL] entries [Tuned OR] [Tuned OFFENSIVE] language",
                     "this sentence contains hateful entries or offensive language",
                    "this sentence contains offensive language"]):

        save_res.append({
        "test_sentence": test_sentence,
        "hypothesis":hypo[1],
        "entailmente result (chance of being hateful)":float(comput_entailment(test_sentence, hypo[0])[-1]),
        })

## Results

In [110]:
pd.DataFrame(save_res)

,test_sentence,hypothesis,entailmente result (chance of being hateful)
0,Pure armenians and georgians are actually gene...,full_optimized,0.735219
1,Pure armenians and georgians are actually gene...,this sentence contains [Tuned HATEFUL] entries...,0.469156
2,Pure armenians and georgians are actually gene...,this sentence contains hateful entries or offe...,0.089403
3,Pure armenians and georgians are actually gene...,this sentence contains offensive language,0.031590
4,She looks like a heshe fag,full_optimized,0.911305
5,She looks like a heshe fag,this sentence contains [Tuned HATEFUL] entries...,0.782841
6,She looks like a heshe fag,this sentence contains hateful entries or offe...,0.269548
7,She looks like a heshe fag,this sentence contains offensive language,0.177524
8,The most recent Miss Universe of Norway is a S...,full_optimized,0.933913
9,The most recent Miss Universe of Norway is a S...,this sentence contains [Tuned HATEFUL] entries...,0.563037
